In [2]:
import pandas

In [3]:
dataset = pandas.read_csv("dataset.tsv",sep="\t")
dataset.head()

,Phylum,Subphylum,Class,Genus,Species,Resource_type,Link,DOI,Publication
0,Cnidaria,Anthozoa,Hexacorallia,Acropora,acuminata,Genome assembly,https://marinegenomics.oist.jp/aacu/viewer/dow...,10.1093/molbev/msaa216,"Shinzato C, Khalturin K, Inoue J, Zayasu Y, Ka..."
1,Cnidaria,Anthozoa,Hexacorallia,Acropora,acuminata,Predicted Genes,https://marinegenomics.oist.jp/aacu/viewer/dow...,10.1093/molbev/msaa216,"Shinzato C, Khalturin K, Inoue J, Zayasu Y, Ka..."
2,Cnidaria,Anthozoa,Hexacorallia,Acropora,awi,Genome assembly,https://marinegenomics.oist.jp/aawi/viewer/dow...,10.1093/molbev/msaa216,"Shinzato C, Khalturin K, Inoue J, Zayasu Y, Ka..."
3,Cnidaria,Anthozoa,Hexacorallia,Acropora,awi,Predicted Genes,https://marinegenomics.oist.jp/aacu/viewer/dow...,10.1093/molbev/msaa216,"Shinzato C, Khalturin K, Inoue J, Zayasu Y, Ka..."
4,Cnidaria,Anthozoa,Hexacorallia,Acropora,cytherea,Genome assembly,https://marinegenomics.oist.jp/acyt/viewer/dow...,10.1093/molbev/msaa216,"Shinzato C, Khalturin K, Inoue J, Zayasu Y, Ka..."


In [6]:

md = ""

Introduction = """# Cnidarian/Ctenophore Resources
This is a collection of links to all datasets of Cnidarian and Ctenophore genome, transcriptome and proteome resources. 
It is unlikely that this is a complete list so if I am missing something please let me know in a email: gideon.bergheim@cos.uni-heidelberg.de.\n\n"""
md += Introduction

def clean(str):
    return str.lower().replace(" ","-")

#toc
for phylum in dataset.Phylum.unique():
    md += "&nbsp;"*4+"[{}](#{})<br>".format(phylum, clean(phylum))
    df_phylum = dataset.loc[dataset.Phylum == phylum]
    for subphylum in df_phylum.Subphylum.unique():
        md += "&nbsp;"*8+"[{}](#{})<br>".format(subphylum, clean(subphylum))
        df_subphylum = df_phylum.loc[df_phylum.Subphylum == subphylum]
        for _class in df_subphylum.Class.unique():
            md += "&nbsp;"*12+"[{}](#{})<br>".format(_class, clean(_class))
            df_class = df_subphylum.loc[df_phylum.Class == _class]
            for genus in df_class.Genus.unique():
                md += "&nbsp;"*16+"[{}](#{})<br>".format(genus, clean(genus))
                df_genus = df_class.loc[df_class.Genus == genus]
                for species in df_genus.Species.unique():
                    df_species = df_genus.loc[df_class.Species == species]
                    md += "&nbsp;"*20+"- [*{}*](#{})<br>".format("{} {}".format(genus,species), clean("{} {}".format(genus,species)))

#data
for phylum in dataset.Phylum.unique():
    phylum_header = "\n## {}\n".format(phylum)
    md += phylum_header
    df_phylum = dataset.loc[dataset.Phylum == phylum]
    for subphylum in df_phylum.Subphylum.unique():
        subphylum_header = "### {}\n".format(subphylum)
        md += subphylum_header
        df_subphylum = df_phylum.loc[df_phylum.Subphylum == subphylum]
        for _class in df_subphylum.Class.unique():
            class_header = "#### {}\n".format(_class)
            md += class_header
            df_class = df_subphylum.loc[df_phylum.Class == _class]
            for genus in df_class.Genus.unique():
                genus_header = "##### {}\n".format(genus)
                md += genus_header
                df_genus = df_class.loc[df_class.Genus == genus]
                for species in df_genus.Species.unique():
                    species_header = "*{} {}*\n".format(genus,species)
                    md += species_header + "\n"

                    df_species = df_genus.loc[df_class.Species == species]
                    for resource in dataset.Resource_type.unique():
                        row = df_species.loc[df_species.Resource_type == resource]
                        if len(row)>0:
                            link = row["Link"].values[0]
                            doi = row["DOI"].values[0]          
                            
                            if not pandas.isnull(row.Publication.item()):
                                pub = row.Publication.item().split(" ")[0] +" et al."
                            else:
                                pub= ""
                            md += "<a href='{}'><img src='icons/{}.svg'></a>".format(link,resource) 
                            md += "- [{}]({})\t\tPublication: {}\t\t DOI: {}\n\n".format(resource,link,pub,doi)
                    md += "<hr class='Species'>\n\n"


p = dataset.Publication.dropna()
for publication in sorted(p.unique()):
    if type(publication) == str:
        s_pub = publication.split(" ")
        formatted = "> **{}** ".format(s_pub[0]) + " ".join(s_pub[1:]) + "\n\n"
        md += formatted

                    

In [7]:
with open("new_README.md","w",encoding="utf-8") as f:
    f.write(md)

&nbsp;&nbsp;&nbsp;&nbsp;[Cnidaria](#cnidaria)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Anthozoa](#anthozoa)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Hexacorallia](#hexacorallia)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Acropora](#acropora)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora acuminata*](#acropora-acuminata)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora awi*](#acropora-awi)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora cytherea*](#acropora-cytherea)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora digitifera*](#acropora-digitifera)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora echinata*](#acropora-echinata)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora florida*](#acropora-florida)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora gemmifera*](#acropora-gemmifera)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora hyacinthus*](#acropora-hyacinthus)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora intermedia*](#acropora-intermedia)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora microphthalma*](#acropora-microphthalma)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora muricata*](#acropora-muricata)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora nasuta*](#acropora-nasuta)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora selago*](#acropora-selago)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora tenuis*](#acropora-tenuis)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Acropora yongei*](#acropora-yongei)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Astreopora](#astreopora)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Astreopora myriophthalma*](#astreopora-myriophthalma)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Porites](#porites)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Porites australiensis*](#porites-australiensis)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Montipora](#montipora)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Montipora cactus*](#montipora-cactus)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Montipora efflorescens*](#montipora-efflorescens)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Medusozoa](#medusozoa)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Scyphozoa](#scyphozoa)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Aurelia](#aurelia)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Aurelia aurita(Atlantic Ocean)*](#aurelia-aurita(atlantic-ocean))<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Aurelia aurita(Roscoff)*](#aurelia-aurita(roscoff))<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Hydrozoa](#hydrozoa)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Hydra](#hydra)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Hydra viridissima*](#hydra-viridissima)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Cubozoa](#cubozoa)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Morbakka](#morbakka)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- [*Morbakka virulenta*](#morbakka-virulenta)<br>
## Cnidaria
### Anthozoa
#### Hexacorallia
##### Acropora
*Acropora acuminata*

- [Genome assembly](https://marinegenomics.oist.jp/aacu/viewer/download?project_id=84 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/aacu/viewer/download?project_id=84 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora awi*

- [Genome assembly](https://marinegenomics.oist.jp/aawi/viewer/download?project_id=85 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/aacu/viewer/download?project_id=85 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora cytherea*

- [Genome assembly](https://marinegenomics.oist.jp/acyt/viewer/download?project_id=86 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/acyt/viewer/download?project_id=86 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora digitifera*

- [Genome assembly](https://marinegenomics.oist.jp/adig/viewer/download?project_id=87 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/adig/viewer/download?project_id=87 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora echinata*

- [Genome assembly](https://marinegenomics.oist.jp/aech/viewer/download?project_id=88 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/aech/viewer/download?project_id=88 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora florida*

- [Genome assembly](https://marinegenomics.oist.jp/aflo/viewer/download?project_id=89 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/aflo/viewer/download?project_id=89 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora gemmifera*

- [Genome assembly](https://marinegenomics.oist.jp/agem/viewer/download?project_id=90 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/agem/viewer/download?project_id=90 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora hyacinthus*

- [Genome assembly](https://marinegenomics.oist.jp/ahya/viewer/download?project_id=91 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/ahya/viewer/download?project_id=91 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora intermedia*

- [Genome assembly](https://marinegenomics.oist.jp/aint/viewer/download?project_id=92 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/aint/viewer/download?project_id=92 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora microphthalma*

- [Genome assembly](https://marinegenomics.oist.jp/amic/viewer/download?project_id=93 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/amic/viewer/download?project_id=93 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora muricata*

- [Genome assembly](https://marinegenomics.oist.jp/amur/viewer/download?project_id=94 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/amur/viewer/download?project_id=94 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora nasuta*

- [Genome assembly](https://marinegenomics.oist.jp/anas/viewer/download?project_id=95 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/anas/viewer/download?project_id=95 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora selago*

- [Genome assembly](https://marinegenomics.oist.jp/asel/viewer/download?project_id=96 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/asel/viewer/download?project_id=96 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora tenuis*

- [Genome assembly](https://marinegenomics.oist.jp/aten/viewer/download?project_id=97 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/aten/viewer/download?project_id=97 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Transcripts](https://marinegenomics.oist.jp/aten/viewer/download?project_id=97 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Proteins](https://marinegenomics.oist.jp/aten/viewer/download?project_id=97 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Transcriptome Assembly](https://marinegenomics.oist.jp/aten/viewer/download?project_id=97 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

*Acropora yongei*

- [Genome assembly](https://marinegenomics.oist.jp/ayon/viewer/download?project_id=98 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

- [Predicted Genes](https://marinegenomics.oist.jp/ayon/viewer/download?project_id=98 )		Publication: Shinzato et al.		 DOI: 10.1093/molbev/msaa216

##### Astreopora
*Astreopora myriophthalma*

- [Genome assembly](https://marinegenomics.oist.jp/amyr_1_1/viewer/download?project_id=107 )		Publication: Yoshioka et al.		 DOI: 10.1186/s12862-022-02023-8

- [Predicted Genes](https://marinegenomics.oist.jp/amyr_1_1/viewer/download?project_id=107 )		Publication: Yoshioka et al.		 DOI: 10.1186/s12862-022-02023-8

##### Porites
*Porites australiensis*

- [Genome assembly](https://marinegenomics.oist.jp/pau/viewer/download?project_id=108)		Publication: Shinzato et al.		 DOI: 10.1093/gbe/evab270

- [Predicted Genes](https://marinegenomics.oist.jp/pau/viewer/download?project_id=108)		Publication: Shinzato et al.		 DOI: 10.1093/gbe/evab270

##### Montipora
*Montipora cactus*

- [Genome assembly](https://marinegenomics.oist.jp/mcac_1_1/viewer/download?project_id=105)		Publication: Yoshioka et al.		 DOI: 10.1186/s12862-022-02023-8

- [Predicted Genes](https://marinegenomics.oist.jp/mcac_1_1/viewer/download?project_id=105)		Publication: Yoshioka et al.		 DOI: 10.1186/s12862-022-02023-8

*Montipora efflorescens*

- [Genome assembly](https://marinegenomics.oist.jp/meff_1_1/viewer/download?project_id=106)		Publication: Yoshioka et al.		 DOI: 10.1186/s12862-022-02023-8

- [Predicted Genes](https://marinegenomics.oist.jp/meff_1_1/viewer/download?project_id=106)		Publication: Yoshioka et al.		 DOI: 10.1186/s12862-022-02023-8

### Medusozoa
#### Scyphozoa
##### Aurelia
*Aurelia aurita(Atlantic Ocean)*

- [Genome assembly](https://marinegenomics.oist.jp/aurelia_aurita/viewer/download?project_id=69)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

- [Predicted Genes](https://marinegenomics.oist.jp/aurelia_aurita/viewer/download?project_id=69)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

- [Predicted Proteins](https://marinegenomics.oist.jp/aurelia_aurita/viewer/download?project_id=69)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

*Aurelia aurita(Roscoff)*

- [Genome assembly](https://marinegenomics.oist.jp/aurelia_aurita_pacific/viewer/download?project_id=75)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

- [Predicted Genes](https://marinegenomics.oist.jp/aurelia_aurita_pacific/viewer/download?project_id=75)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

- [Predicted Proteins](https://marinegenomics.oist.jp/aurelia_aurita_pacific/viewer/download?project_id=75)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

#### Hydrozoa
##### Hydra
*Hydra viridissima*

- [Genome assembly](https://marinegenomics.oist.jp/hydra_viridissima_a99/viewer/download?project_id=82)		Publication: 		 DOI: nan

- [Predicted Genes](https://marinegenomics.oist.jp/hydra_viridissima_a99/viewer/download?project_id=82)		Publication: 		 DOI: nan

- [Predicted Proteins](https://marinegenomics.oist.jp/hydra_viridissima_a99/viewer/download?project_id=82)		Publication: 		 DOI: nan

#### Cubozoa
##### Morbakka
*Morbakka virulenta*

- [Genome assembly](https://marinegenomics.oist.jp/morbakka_virulenta/viewer/download?project_id=70)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

- [Predicted Genes](https://marinegenomics.oist.jp/morbakka_virulenta/viewer/download?project_id=70)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

- [Predicted Proteins](https://marinegenomics.oist.jp/morbakka_virulenta/viewer/download?project_id=70)		Publication: Khalturin, et al.		 DOI: 10.1038/s41559-019-0853-y

> **Shinzato** C, Khalturin K, Inoue J, Zayasu Y, Kanda M, Kawamitsu M, Yoshioka Y, Yamashita H, Suzuki G, Satoh N. Eighteen coral genomes reveal the evolutionary origin of Acropora strategies to accommodate environmental changes. Molecular Biology and Evolution (2020) Doi: 10.1093/molbev/msaa216

> **Yoshioka** Y, Suzuki G, Zayasu Y, Yamashita H, Shinzato C. Comparative genomics highlight the importance of lineage-specific gene families in evolutionary divergence of the coral genus, Montipora BMC Ecol Evol. 22(1):71.

> **Khalturin,** K., Shinzato, C., Khalturina, M., Hamada, M., Fujie, M., Koyanagi, R., Kanda, M., Goto, H., Anton-Erxleben, F., Toyokawa, M., Toshino, S., Satoh, N. Medusozoan genomes inform the evolution of the jellyfish body plan Nature Ecology & Evolution

> **Khalturin,** K., Shinzato, C., Khalturina, M. et al. Medusozoan genomes inform the evolution of the jellyfish body plan. Nat Ecol Evol 3, 811–822 (2019). https://doi.org/10.1038/s41559-019-0853-y

> **Shinzato** C, Takeuchi T, Yoshioka Y, Tada I, Kanda M, Broussard C, Iguchi A, Kusakabe M, Marin F, Satoh N, Inoue M. Whole-Genome Sequencing Highlights Conservative Genomic Strategies of a Stress-Tolerant, Long-Lived Scleractinian Coral, Porites australiensis Vaughan, 1918. Genome Biol Evol. 2021 Dec 1;13(12):evab270. doi: 10.1093/gbe/evab270. PMID: 34878117; PMCID: PMC8691061.

> **Yoshioka** Y, Suzuki G, Zayasu Y, Yamashita H, Shinzato C. Comparative genomics highlight the importance of lineage-specific gene families in evolutionary divergence of the coral genus, Montipora. BMC Ecol Evol. 2022 May 27;22(1):71. doi: 10.1186/s12862-022-02023-8. PMID: 35624412; PMCID: PMC9145168.
